In [148]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join


import duckdb #pip install duckdb==0.9.1
import plotly.express as px #pip install plotly_express==0.4.0

from datetime import datetime

# 1. \movielens_latest_datasets
https://files.grouplens.org/datasets/movielens/ml-latest-README.html

In [149]:
listdir('файлы для тестов/recsys_project/dataset')

['genome-scores.csv',
 'genome-tags.csv',
 'links.csv',
 'movies.csv',
 'ratings.csv',
 'README.txt',
 'tags.csv']

In [150]:
df_ratings = pd.read_csv(f'файлы для тестов/movielens_latest_datasets/ratings.csv')
df_ratings.name = 'ratings'

df_movies = pd.read_csv(f'файлы для тестов/movielens_latest_datasets/movies.csv')
df_movies.name = 'movies'

df_tags = pd.read_csv(f'файлы для тестов/movielens_latest_datasets/tags.csv')
df_tags.name = 'tags'

df_links = pd.read_csv(f'файлы для тестов/movielens_latest_datasets/links.csv')
df_links.name = 'links'

df_genome_tags = pd.read_csv(f'файлы для тестов/movielens_latest_datasets/genome-tags.csv')
df_genome_tags.name = 'genome_tags'

df_genome_scores = pd.read_csv(f'файлы для тестов/movielens_latest_datasets/genome-scores.csv')
df_genome_scores.name = 'genome_scores'

# 3.1. Валидность данных (количество пропусков, количество уникальных значений)

In [151]:
df_list = [df_ratings,df_movies,df_tags,df_links,df_genome_tags,df_genome_scores]
list_val = [] 

for df_i in df_list:
    for df_col in df_i.columns:
        all_el = len(df_i)
        uniq_el = df_i[df_col].nunique()
        null_el = len(df_i[df_i[df_col].isnull()])
        list_val.append([df_i.name,df_col,all_el,uniq_el,null_el])
        
df_valid = pd.DataFrame(list_val, columns=['name_df','name_col','cnt_all', 'cnt_iniq', 'cnt_null'])
df_valid['cnt_iniq_perc'] = round((df_valid.cnt_iniq / df_valid.cnt_all)*100,1)
df_valid

,name_df,name_col,cnt_all,cnt_iniq,cnt_null,cnt_iniq_perc
0,ratings,userId,33832162,330975,0,1.0
1,ratings,movieId,33832162,83239,0,0.2
2,ratings,rating,33832162,10,0,0.0
3,ratings,timestamp,33832162,27419646,0,81.0
4,movies,movieId,86537,86537,0,100.0
5,movies,title,86537,86330,0,99.8
6,movies,genres,86537,1796,0,2.1
7,tags,userId,2328315,25280,0,1.1
8,tags,movieId,2328315,53452,0,2.3
9,tags,tag,2328315,153949,17,6.6


# 3.3. Определение самых популярных фильмов (по оценке, ко количеству просмотров)

In [152]:
t_ratings = df_ratings.copy()
t_movies = df_movies.copy()

q = '''
SELECT tm.title, tr.movieId, count(tr.userId) as cnt_view, mean(tr.rating) as mean_rating
FROM t_ratings as tr
LEFT JOIN t_movies as tm
ON tr.movieId = tm.movieId
GROUP BY 1,2
ORDER BY 3 desc
limit 100
'''
t_fin = duckdb.query(q).to_df()

fig = px.scatter(t_fin,
                 x="cnt_view",
                 y="mean_rating",
                 size="cnt_view",
                 #color="continent",
                 hover_name="title",
                 log_x=True,
                 size_max=60)
fig.show()

# 3.5. Количество выпущенных фильмов (по годам). Посмотреть, какие жанры наиболее популярны в последние годы (что чаще снимают: мелодрамы или боевики)

In [153]:
t1 = t_movies.copy()

# извлекаем год
t1['year_movie'] = t1['title'].str.extract(r'\((\d{4})\)')
t1['year_movie'] = pd.to_numeric(t1['year_movie'], errors='coerce')
t1 = t1.dropna(subset=['year_movie'])
t1['year_movie'] = t1['year_movie'].astype(int)

# Используем функцию explode() для разделения значений в столбце 'genres' на отдельные строки
t1['genres'] = t1['genres'].str.split('|')
t1 = t1.explode('genres')


# t1['gn_action'] = np.where(t1.genres.str.contains('Action', regex=False), 1, 0)

In [154]:
q = '''
SELECT
    year_movie,
    genres,
    COUNT(title) as cnt
FROM t1
GROUP BY 1,2
ORDER BY 1,2
'''
t_fin = duckdb.query(q).to_df()

fig = px.line(t_fin,
              x="year_movie",
              y="cnt",
              color="genres",
              hover_name="genres",
              line_shape="linear",
              #render_mode="svg"
             )
fig.show()

In [155]:
# t1 - фильмы
# t_ratings - оценки

q = '''
with cte0 as(
SELECT
    tr.userId,
    tr.rating,
    tm.title,
    tm.year_movie,
    tm.genres
FROM t_ratings as tr
LEFT JOIN t1 as tm
ON tr.movieId = tm.movieId
WHERE tm.title IS NOT NULL
)
SELECT
    year_movie,
    genres,
    count(userId) as cnt_view,
    mean(rating) as mean_rating
FROM cte0
GROUP BY 1,2
ORDER BY 1,2
'''
t_fin = duckdb.query(q).to_df()

fig = px.scatter(t_fin,
                 x="year_movie",
                 y="mean_rating",
                 size="cnt_view",
                 color="genres",
                 hover_name="genres",
                 log_x=True,
                 size_max=60)
fig.show()

# 3.7. Можно посмотреть, какие фильмы чаще оценивают в какие месяцы (например пользователь под Новый год может захотеть посмотреть один жанр, под майские праздники другой. Попробовать уловить сезонность

In [156]:
t_ratings['dt'] = pd.to_datetime(t_ratings['timestamp'], unit='s').dt.year
t_ratings['dt_mnth'] = pd.to_datetime(t_ratings['timestamp'], unit='s').dt.month
t_ratings.head()

,userId,movieId,rating,timestamp,dt,dt_mnth
0,1,1,4.0,1225734739,2008,11
1,1,110,4.0,1225865086,2008,11
2,1,158,4.0,1225733503,2008,11
3,1,260,4.5,1225735204,2008,11
4,1,356,5.0,1225735119,2008,11


In [157]:
q = '''
with cte0 as(
    SELECT tr.*, t1.title, t1.genres, t1.year_movie
    from t_ratings as tr
    left join t1
    on tr.movieId = t1.movieId
), cte1 as(
    select dt, dt_mnth, genres, count(userId) as cnt_view
    from cte0
    group by 1,2,3
    order by 1,2,3
), cte2 as(
    select *,
        sum(cnt_view) over(partition by dt,dt_mnth) as mnth_sum
    from cte1
)
select
    dt,
    dt_mnth,
    genres,
    cnt_view,
    round(cnt_view::decimal/mnth_sum::decimal,3)*100 as perc_view_mnth
from cte2

'''
t_fin = duckdb.query(q).to_df()
t_fin.head()

,dt,dt_mnth,genres,cnt_view,perc_view_mnth
0,1999,1,Action,7339,10.5
1,1999,1,Adventure,5526,7.9
2,1999,1,Animation,1203,1.7
3,1999,1,Children,2458,3.5
4,1999,1,Comedy,10400,14.8


In [158]:
import plotly.express as px

fig22 = px.bar(t_fin[t_fin.dt == 2022], x="dt_mnth", y="perc_view_mnth", color="genres", title="2022")
fig20 = px.bar(t_fin[t_fin.dt == 2020], x="dt_mnth", y="perc_view_mnth", color="genres", title="2020")
fig18 = px.bar(t_fin[t_fin.dt == 2018], x="dt_mnth", y="perc_view_mnth", color="genres", title="2018")
fig10 = px.bar(t_fin[t_fin.dt == 2010], x="dt_mnth", y="perc_view_mnth", color="genres", title="2010")
fig05 = px.bar(t_fin[t_fin.dt == 2005], x="dt_mnth", y="perc_view_mnth", color="genres", title="2005")

fig22.show()
fig20.show()
fig18.show()
fig10.show()
fig05.show()

# 3.9. Посмотреть количество уникальных пользователей по годам

In [159]:
t_ratings.head()

,userId,movieId,rating,timestamp,dt,dt_mnth
0,1,1,4.0,1225734739,2008,11
1,1,110,4.0,1225865086,2008,11
2,1,158,4.0,1225733503,2008,11
3,1,260,4.5,1225735204,2008,11
4,1,356,5.0,1225735119,2008,11


In [160]:
t_ratings[t_ratings.dt == 1996].userId.nunique()

34448

In [161]:
q = '''
with cte0 as(
    select dt, userId, count(rating) as cnt_rate
    from t_ratings
    group by 1,2
)
select dt,
    count(userId) as cnt_uniq_user,
    avg(cnt_rate) as mean_rate_per_user
from cte0
group by 1
order by 1
'''
t_fin = duckdb.query(q).to_df()


fig = px.line(t_fin, x="dt", y="cnt_uniq_user",hover_name="cnt_uniq_user")
fig.show()

fig = px.line(t_fin, x="dt", y="mean_rate_per_user",hover_name="mean_rate_per_user")
fig.show()

# 3.11. Попробовать по тегам определить, что влияет на рейтинг фильма

df_ratings     
df_movies    
df_tags    
df_links    
df_genome_tags    
df_genome_scores    

### ПОЛЬЗОВАТЕЛЬСКИЕ ТЕГИ

In [162]:
t_ratings_for_tags = df_ratings.copy()
t_ratings_for_tags['cnt_rate'] = t_ratings_for_tags.groupby('movieId')['userId'].transform('count')
t_ratings_for_tags = t_ratings_for_tags[t_ratings_for_tags.cnt_rate >= 50]

t_ratings_for_tags.head()

,userId,movieId,rating,timestamp,cnt_rate
0,1,1,4.0,1225734739,76813
1,1,110,4.0,1225865086,75514
2,1,158,4.0,1225733503,16591
3,1,260,4.5,1225735204,97202
4,1,356,5.0,1225735119,113581


In [163]:
t_ratings_for_tags.movieId.nunique()

16116

In [164]:
t_ratings_for_tags = t_ratings_for_tags[['movieId','cnt_rate', 'rating']].groupby(['movieId','cnt_rate'])\
.agg('mean')\
.reset_index()

t_ratings_for_tags

,movieId,cnt_rate,rating
0,1,76813,3.893508
1,2,30209,3.278179
2,3,15820,3.171271
3,4,3028,2.868395
4,5,15801,3.076957
...,...,...,...
16111,287377,92,2.815217
16112,287633,145,3.620690
16113,288167,105,3.376190
16114,288265,89,3.775281


In [165]:
t_tags = df_tags.merge(t_ratings_for_tags, on=['movieId'], how='left')
t_tags.head()

,userId,movieId,tag,timestamp,cnt_rate,rating
0,10,260,good vs evil,1430666558,97202.0,4.09240
1,10,260,Harrison Ford,1430666505,97202.0,4.09240
2,10,260,sci-fi,1430666538,97202.0,4.09240
3,14,1221,Al Pacino,1311600756,47271.0,4.26951
4,14,1221,mafia,1311600746,47271.0,4.26951


In [166]:
q = '''
select tag, avg(rating) as rate_tag
from t_tags
group by 1
order by 2 desc
'''
t_fin = duckdb.query(q).to_df()

# Установка опции для вывода всех строк
pd.set_option('display.max_rows', 100)

t_fin.head(100)

,tag,rate_tag
0,Komodo dragon,4.451739
1,Namib,4.451739
2,macaque,4.451739
3,Jaipur,4.451739
4,langur,4.451739
5,antonishing,4.451218
6,impossible shots,4.449308
7,REALISTIC,4.423986
8,BASED ON TRUE STORY,4.423986
9,serie,4.423986


In [167]:
q = '''
select tag, avg(rating) as rate_tag
from t_tags
group by 1
order by 2
'''
t_fin = duckdb.query(q).to_df()

# Установка опции для вывода всех строк
pd.set_option('display.max_rows', 100)

t_fin.head(100)

,tag,rate_tag
0,would kill not to watch,1.011247
1,warning: some people tryed to kill themselves ...,1.011247
2,Hilarious audio commentary,1.011247
3,movie hell,1.011247
4,death on screen,1.011247
5,worst acting,1.011247
6,Guarini is a goon,1.011247
7,can't ever rate it - cuz I can't ever watch th...,1.214450
8,Unromantic,1.214450
9,Insipid dialogue,1.214450


### ГЕНОМНЫЕ ТЕГИ

In [168]:
tags_genome = df_genome_scores.merge(df_genome_tags, on=['tagId'], how='left')
tags_genome = tags_genome[tags_genome.relevance > 0.5][['movieId','tag']]

tags_genome = tags_genome.merge(t_ratings_for_tags, on=['movieId'], how='left')
tags_genome.head()

,movieId,tag,cnt_rate,rating
0,1,3d,76813.0,3.893508
1,1,action,76813.0,3.893508
2,1,adventure,76813.0,3.893508
3,1,affectionate,76813.0,3.893508
4,1,animal movie,76813.0,3.893508


In [169]:
q = '''
select tag, avg(rating) as rate_tag
from tags_genome
group by 1
order by 2 desc
'''
t_fin = duckdb.query(q).to_df()

# Установка опции для вывода всех строк
pd.set_option('display.max_rows', 100)

t_fin.head(100)

,tag,rate_tag
0,brilliant,4.167204
1,perfect,4.142499
2,awesome,4.009345
3,afi 100,3.997308
4,afi 100 (laughs),3.905890
5,miyazaki,3.889671
6,afi 100 (movie quotes),3.881327
7,studio ghibli,3.873108
8,intelligent,3.871892
9,moving,3.868978


In [170]:
q = '''
select tag, avg(rating) as rate_tag
from tags_genome
group by 1
order by 2
'''
t_fin = duckdb.query(q).to_df()

# Установка опции для вывода всех строк
pd.set_option('display.max_rows', 100)

t_fin.head(100)

,tag,rate_tag
0,overrated,1.909736
1,awful,2.037742
2,bad,2.188662
3,boring,2.205764
4,lame,2.234783
5,unintentionally funny,2.319826
6,bad script,2.340709
7,so bad it's good,2.353771
8,horrible,2.374204
9,bad sequel,2.392245


# 3.13. Распределение positive/negative в разрезе рейтинга по тегам

# 3.16. Вывести статистики в процентном соотношении - сколько оценок в год

In [171]:
t_ratings['sum_per_dt'] = t_ratings.groupby('dt')['userId'].transform('count')
t_ratings['sum_dt_all'] = len(t_ratings)

q = '''
select dt, round(sum_per_dt::decimal/sum_dt_all::decimal,3)*100 as perc_rate
from(SELECT distinct dt, sum_per_dt, sum_dt_all FROM t_ratings) as t0
ORDER BY 1
'''
t_fin = duckdb.query(q).to_df()

fig = px.line(t_fin,
              x="dt",
              y="perc_rate",
              #color="genres",
              hover_name="perc_rate",
              #line_shape="linear",
              #render_mode="svg"
             )
fig.show()

t2 = t1[t1['genres'].isin(['Thriller','Sci-Fi','Fantasy','Drama','Crime','Comedy','Adventure','Action'])]
t2.head()

# 3.17. Вывести статистики в процентном соотношении - сколько оценок в год в год выпуска фильма

In [172]:
t1_uniq = t1[['movieId','title','year_movie']].drop_duplicates()
t1_uniq.head()

,movieId,title,year_movie
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [173]:
ratings_movies_year = t_ratings.merge(t1_uniq, on='movieId', how='left')
ratings_movies_year = ratings_movies_year[ratings_movies_year.dt==ratings_movies_year.year_movie]

ratings_movies_year['sum_per_dt'] = ratings_movies_year.groupby('dt')['userId'].transform('count')
ratings_movies_year['sum_dt_all'] = len(ratings_movies_year)

ratings_movies_year.head()

,userId,movieId,rating,timestamp,dt,dt_mnth,sum_per_dt,sum_dt_all,title,year_movie
80,2,95,2.0,835816548,1996,6,141223,1160327,Broken Arrow (1996),1996.0
83,2,141,5.0,835816548,1996,6,141223,1160327,"Birdcage, The (1996)",1996.0
152,2,786,4.0,835817123,1996,6,141223,1160327,Eraser (1996),1996.0
181,3,185029,5.0,1536174197,2018,9,42348,1160327,A Quiet Place (2018),2018.0
182,3,187593,3.0,1536174244,2018,9,42348,1160327,Deadpool 2 (2018),2018.0


In [174]:
q = '''
with cte0 as(
    select *
    from ratings_movies_year
    where dt::int = year_movie::int
)
select dt, round(sum_per_dt::decimal/sum_dt_all::decimal,3)*100 as perc_rate
from(SELECT distinct dt, sum_per_dt, sum_dt_all FROM cte0) as t0
ORDER BY 1
'''


t_fin = duckdb.query(q).to_df()

fig = px.line(t_fin,
              x="dt",
              y="perc_rate",
              #color="genres",
              hover_name="perc_rate",
              #line_shape="linear",
              #render_mode="svg"
             )
fig.show()

In [175]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [177]:
import plotly.io as pio

pio.write_html(fig, file='figure.html', auto_open=True)

In [178]:
{% include figure.html %}

SyntaxError: invalid syntax (1074604756.py, line 1)